In [1]:
import pandas as pd
import numpy as np
import collections
import time
import math

In [2]:
item_metadata = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_all.csv',
                           sep=',',
                           usecols=['item_id','n_clicks','views','clicks_views_ratio','sum_of_properties'])
item_metadata[:3]

,item_id,n_clicks,views,clicks_views_ratio,sum_of_properties
0,5001,3.0,99,0.030303,48.0
1,5002,6.0,68,0.088235,55.0
2,5003,6.0,90,0.066667,73.0


In [101]:
train = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\train.csv',
                       sep=',',
                    skiprows=range(1, 5000000)
#                    nrows=1000000
                   )
train = train[train["action_type"] == "clickout item"]
print(train.shape)
train[:3]

(764833, 13)


,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
4,5000003,aae8bb89fd351,1541202599,FBDSXTDETR9A,3,clickout item,48429,BR,"Paris, France",desktop,NaN,48429|49317|11898|48512|82442|49759|17905|1197...,80|82|84|107|100|70|109|100|181|86|48|96|175|8...
5,5000004,6dd2e491d74e4,1541202599,BLCN13KY2O6W,88,clickout item,8275678,CA,"Port Campbell, Australia",mobile,NaN,1577593|1054282|8275678|97157|1147613|1370793|...,103|144|135|96|152|69|181|109|83|325
7,5000006,02006b3a59013,1541202599,87VRAF01GR1N,8,clickout item,2600412,MY,"Malacca, Malaysia",mobile,NaN,2600728|2026377|3135188|888779|2547593|4174444...,40|33|42|82|59|73|25|100|49|195|201|63|27|23|7...


In [102]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_impressions_and_prices(df_in):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, 'impressions'] = df['impressions'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]
    df.loc[:, 'prices'] = df['prices'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

    df = df.sort_values(by=['session_id'])
    df['number_of_impressions'] = df['impressions'].apply(len)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df['impressions'].str.len())
         for col in df.columns.drop('impressions')}
    )
    

    df_out.loc[:, 'impressions'] = np.concatenate(df['impressions'].values)
    df_out.loc[:, 'impressions'] = df_out['impressions'].apply(int)
    
    df_out.loc[:, 'prices'] = np.concatenate(df['prices'].values)
    df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

    return df_out

In [103]:
train = explode_impressions_and_prices(train)
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126


In [104]:
train['position'] = train.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306,1
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126,2
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126,3


In [105]:
# # DODANIE CITY I COUNTRY
# city, country = (train['city']
#                     .values[0]
#                     .replace(" ", "")
#                     .split(','))
# print(city)
# print(country)

# cities = []
# countries = []
# l = []
# for index in range(len(train['city'].values)):
#     l = (train['city']
#                     .values[index]
#                     .replace(" ", "")
#                     .split(','))
#     cities.append(l[0])
#     countries.append(l[1])
# #     if len(l) > 2 :
# #         print(l[0])
# #         print(l[1])
# #         print(l[2])
# #         print(len(l))

# train = train.drop(['city'], axis=1)
# train['city'] = cities
# train['country'] = countries
# train[:3]

In [106]:
def mean_price(df):
    return df['prices'].mean()
df_mean_price = train.groupby(['session_id', 'timestamp']).apply(lambda df: df['prices'].mean()).to_frame("mean_price")
df_mean_price[:3]
train = train.merge(df_mean_price,
                     left_on=['session_id', 'timestamp'],
                     right_on=['session_id', 'timestamp'],
                     how='left')
train['mean_price_difference'] = abs(train['prices'] - train['mean_price'])
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position,mean_price,mean_price_difference
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306,1,82.6,13.4
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126,2,82.6,1.6
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126,3,82.6,4.4


In [107]:
train = train.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,...,number_of_impressions,impressions,position,mean_price,mean_price_difference,item_id,n_clicks,views,clicks_views_ratio,sum_of_properties
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,25,5479306,1,82.6,13.4,5479306,2.0,38,0.052632,89.0
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,25,97126,2,82.6,1.6,97126,23.0,304,0.075658,72.0
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,25,2195126,3,82.6,4.4,2195126,20.0,179,0.111732,75.0


In [108]:
train = train.rename({'index':'item_id'})
train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','sum_of_properties','mean_price','mean_price_difference','position']]
train['reference'] = train['reference'].astype(int)
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,sum_of_properties,mean_price,mean_price_difference,position
0,5479306,5479306,2.0,38,0.052632,89.0,82.6,13.4,1
1,97126,5479306,23.0,304,0.075658,72.0,82.6,1.6,2
2,2195126,5479306,20.0,179,0.111732,75.0,82.6,4.4,3


In [109]:
train['clicked'] = np.where(train['item_id'] == train['reference'], 1, 0)
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,sum_of_properties,mean_price,mean_price_difference,position,clicked
0,5479306,5479306,2.0,38,0.052632,89.0,82.6,13.4,1,1
1,97126,5479306,23.0,304,0.075658,72.0,82.6,1.6,2,0
2,2195126,5479306,20.0,179,0.111732,75.0,82.6,4.4,3,0


# NAUKA

In [110]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, normalize
from imblearn.under_sampling import RandomUnderSampler

In [129]:
# 'n_clicks','views','clicks_views_ratio', 'mean_price','mean_price_difference','position'
X_train, X_test, y_train, y_test = train_test_split(train[['clicks_views_ratio','position']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
# rus = RandomUnderSampler(random_state=0)
# X_train, y_train = rus.fit_resample(X_train, y_train)

In [130]:
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
# X_train = normalize(X_train, norm='l2')
X_test = sc.transform(X_test)
# X_test = normalize(X_test, norm='l2')

In [133]:
# logreg = DecisionTreeClassifier(class_weight={0.0:0.8, 1.0:0.2})
logreg = LogisticRegression(n_jobs=-1, class_weight={0.0:0.1, 1.0:0.9}) #class_weight={0.0:0.2, 1.0:0.8}
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
# y_pred_prob = logreg.predict_proba(X_test)

D:\Instalowane\Programy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Instalowane\Programy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [134]:
print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

Counter({0: 3359256, 1: 153188})
Counter({1: 3214819, 0: 297625})
balanced_accuracy_score: 0.542237908989275
accuracy_score: 0.12800346425451908
Nieprawidłowo sklasyfikowane próbki: 3062839
classification_report :
               precision    recall  f1-score   support

           0       1.00      0.09      0.16   3359256
           1       0.05      1.00      0.09    153188

    accuracy                           0.13   3512444
   macro avg       0.52      0.54      0.13   3512444
weighted avg       0.96      0.13      0.16   3512444

[[ 297021 3062235]
 [    604  152584]]


# ZBIÓR TESTOWY

In [115]:
test = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\test.csv',
                       sep=',',
#                     skiprows=range(1, 11000000)
#                    nrows=10000
                   )
# test = test[test["action_type"] == "clickout item" & test['reference'].isnull()]
mask = test["reference"].isnull() & (test["action_type"] == "clickout item")
test = test[mask]
# print(test.shape)
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
9,U9MAV55LZMOR,1541453209,0dd5544c71728,1,clickout item,NaN,US,"Atlantic City, USA",mobile,NaN,366186|56563|3741160|63568|3478738|3137008|678...,69|68|46|39|69|47|48|52|74|70|44|34|57|69|37|7...
15,VXIWZ3NF0WHF,1541453212,be1b5623b2e12,3,clickout item,NaN,DK,"Copenhagen, Denmark",tablet,NaN,4412788|53876|3147220|929429|148127|53835|1753...,122|144|195|100|92|117|72|107|268|166|171|657|...
16,2VZWKE7NWP8C,1541453213,7daf68934bfef,1,clickout item,NaN,US,"Weston, USA",mobile,NaN,57189|404846|164156|111309|148435|77409|103228...,115|166|115|97|111|87|200|101|114|160|98|97|10...


In [116]:
test = explode_impressions_and_prices(test)
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016


In [117]:
test['position'] = test.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810,1
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141,2
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016,3


In [118]:
index_df = test[["user_id","session_id","timestamp","step","impressions"]]
index_df[:10]

,user_id,session_id,timestamp,step,impressions
0,0TBXPQCK401O,000056cd97ce2,1541484101,4,3811810
1,0TBXPQCK401O,000056cd97ce2,1541484101,4,496141
2,0TBXPQCK401O,000056cd97ce2,1541484101,4,674016
3,0TBXPQCK401O,000056cd97ce2,1541484101,4,1153426
4,0TBXPQCK401O,000056cd97ce2,1541484101,4,1339848
5,0TBXPQCK401O,000056cd97ce2,1541484101,4,8400372
6,0TBXPQCK401O,000056cd97ce2,1541484101,4,693816
7,0TBXPQCK401O,000056cd97ce2,1541484101,4,521336
8,0TBXPQCK401O,000056cd97ce2,1541484101,4,519061
9,0TBXPQCK401O,000056cd97ce2,1541484101,4,1482717


In [119]:
test = test.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position,item_id,n_clicks,views,clicks_views_ratio,sum_of_properties
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810,1,3811810.0,238.0,1946.0,0.122302,48.0
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141,2,496141.0,258.0,1951.0,0.132240,76.0
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016,3,674016.0,93.0,2184.0,0.042582,46.0


In [120]:
test = test.rename({'index':'item_id'})
test = test[["item_id", "reference",'n_clicks','views','clicks_views_ratio', 'position']]
test = test.drop(columns=['reference'], axis=1)
test = test.dropna()
test[:3]

,item_id,n_clicks,views,clicks_views_ratio,position
0,3811810.0,238.0,1946.0,0.122302,1
1,496141.0,258.0,1951.0,0.132240,2
2,674016.0,93.0,2184.0,0.042582,3


In [121]:
X_test = test[['clicks_views_ratio', 'position']]
X_test = sc.transform(X_test)

In [122]:
y_pred = logreg.predict_proba(X_test)

In [123]:
y_pred

array([[0.1233176 , 0.8766824 ],
       [0.11906496, 0.88093504],
       [0.37218236, 0.62781764],
       ...,
       [0.90037682, 0.09962318],
       [0.82756729, 0.17243271],
       [0.76097131, 0.23902869]])

In [124]:
result = pd.concat([index_df, pd.DataFrame(y_pred)], axis=1, sort=False)
result = result.drop([0], axis=1)
result[:10]

,user_id,session_id,timestamp,step,impressions,1
0,0TBXPQCK401O,000056cd97ce2,1541484101,4,3811810,0.876682
1,0TBXPQCK401O,000056cd97ce2,1541484101,4,496141,0.880935
2,0TBXPQCK401O,000056cd97ce2,1541484101,4,674016,0.627818
3,0TBXPQCK401O,000056cd97ce2,1541484101,4,1153426,0.812202
4,0TBXPQCK401O,000056cd97ce2,1541484101,4,1339848,0.486493
5,0TBXPQCK401O,000056cd97ce2,1541484101,4,8400372,0.520322
6,0TBXPQCK401O,000056cd97ce2,1541484101,4,693816,0.517949
7,0TBXPQCK401O,000056cd97ce2,1541484101,4,521336,0.429805
8,0TBXPQCK401O,000056cd97ce2,1541484101,4,519061,0.462476
9,0TBXPQCK401O,000056cd97ce2,1541484101,4,1482717,0.355299


In [125]:
result = result.assign(impressions=lambda x: x["impressions"].apply(str))
result = result.sort_values(["user_id", "session_id", "timestamp", "step", 1],
                     ascending=[True, True, True, True, False])
result[:10]

,user_id,session_id,timestamp,step,impressions,1
2958346,000I430EXZC0,ddabdf53a4d38,1541501642,1,3247218,0.804236
2958347,000I430EXZC0,ddabdf53a4d38,1541501642,1,8805412,0.682383
2958351,000I430EXZC0,ddabdf53a4d38,1541501642,1,103658,0.680367
2958352,000I430EXZC0,ddabdf53a4d38,1541501642,1,103670,0.589692
2958356,000I430EXZC0,ddabdf53a4d38,1541501642,1,2111996,0.589387
2958350,000I430EXZC0,ddabdf53a4d38,1541501642,1,98265,0.573076
2958349,000I430EXZC0,ddabdf53a4d38,1541501642,1,95011,0.569801
2958348,000I430EXZC0,ddabdf53a4d38,1541501642,1,103452,0.520182
2958353,000I430EXZC0,ddabdf53a4d38,1541501642,1,321771,0.474324
2958354,000I430EXZC0,ddabdf53a4d38,1541501642,1,1148728,0.458059


In [126]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out

In [127]:
result = group_concat(result, ["user_id","session_id","timestamp","step"], "impressions")
result = result.rename(columns={"impressions":"item_recommendations"})
result[:10]

,user_id,session_id,timestamp,step,item_recommendations
0,000I430EXZC0,ddabdf53a4d38,1541501642,1,3247218 8805412 103658 103670 2111996 98265 95...
1,000O8CPC6T0W,846b712ae8e1f,1541480658,1,5961668 6506 51012 52993 923037 6221916 149270...
2,000REY2YYH1D,4a313f2eb8954,1541465923,1,355861 65392 65491 62474 67551 57541 70266 569...
3,0010PD3FH7SV,26b9e7e18cc87,1541535269,2,1709889 6345002 10368292 9654292 1335949 21626...
4,0012IHLNTQWC,a035e580f0e64,1541505846,39,2502093 1307328 1515725 2659342 927367 4079384...
5,0015CRBC6OH9,8bcbd478ac2cf,1541519175,3,48460 2179674 153360 83359 12098 1540815 50270...
6,0018EAWF0OIK,10ce4a98a476f,1541513087,2,8230426 4072668 4722096 3804510 9219290 789271...
7,001EBIZM062K,0e036884adc53,1541502068,1,42276 42262 42134 42291 2619788 9866100 480817...
8,001G88F8NP9P,2a2e9eb9c967c,1541511909,9,513106 3165837 7125170 1814133 4423678 7202328...
9,001JFV0AYE6Z,73ad1e08b4430,1541481561,1,67196 596076 5091882 149620 61991 59822 58414 ...


In [128]:
result.to_csv("D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\submission_ML.csv", index=False,)